https://sciforum.net/manuscripts/5158/slides.pdf

In [1]:
import xarray
import numpy
import matplotlib.pyplot as plt
import pandas
import geopandas
import math

In [2]:
tiles = ['11SLB', # UCSB
         '10SFH', # LNBL 1
         '10SGH', # LNBL 2         
         '11SQD', # DRI 1
         '11SQC', # DRI 2
         '11TNK', # BSU 1
         '11TNJ', # BSU 2
         '11TPK', # BSU 3
         '11TPJ'  # BSU 5
        ]

#tiles = ['10SFH']

In [3]:
prefix = '/data/sentinel2/'

In [4]:
def read_10m(tile):
    zarr_store = f'{prefix}/zarrs/{tile}_10m.zarr'
    print(zarr_store)
    ds_10m = xarray.open_zarr(zarr_store)
    ds_10m = ds_10m.drop_vars('spatial_ref') # Only keep it for either 10m or 20m
    ds_10m = ds_10m.sel(band=['B2', 'B3','B4', 'B8'])
    return ds_10m

def read_20m(tile):
    zarr_store = f'{prefix}/zarrs/{tile}_20m.zarr'
    ds_20m = xarray.open_zarr(zarr_store)
    ds_20m = ds_20m.sel(band=['B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
    return ds_20m

def read_60m(tile):
    zarr_store = f'{prefix}/zarrs/{tile}_60m.zarr'
    ds_60m = xarray.open_zarr(zarr_store)
    ds_60m = ds_60m.sel(band=['B1', 'B9'])
    return ds_60m

def upscale(ds_10m, ds_20m):
    """
    We might feel tempted to exclude the first and last values in x and y (because become NaNs)
    But that would mean that the shape and thus chunk size would change
    """
    ds_20m_upscale = ds_20m.interp(x=ds_10m.x,
                                   y=ds_10m.y,
                                   method='nearest',
                                   method_non_numeric='pad')
    return ds_20m_upscale

def sharpen1(ds_10m, ds_20m_upscale):
    # Method 1: Sum across
    panband = ds_10m['reflectance'].sum(dim=('band')) / 4
    intensity = ds_20m_upscale['reflectance'].sum(dim='band') / 6
    sharpend = (ds_20m_upscale['reflectance'] / intensity * panband )
    return sharpend

def sharpen2(ds_10m, ds_20m_upscale):
    # Method 2: Excluse the SWIR bands
    panband = ds_10m['reflectance'].sum(dim=('band')) / 4
    intensity = ds_20m_upscale['reflectance'].sel(band=['B5', 'B6', 'B7', 'B8A']).sum(dim='band') / 4
    sharpend = (ds_20m_upscale['reflectance'] / intensity * panband )
    return sharpend

def sharpen3(ds_10m, ds_20m_upscale):
    # Method 3: Scale on band 8/8A only
    panband = ds_10m['reflectance'].sel(band='B8')
    intensity = ds_20m_upscale['reflectance'].sel(band='B8A')
    intensity = intensity.where(intensity!=0)
    sharpend = (ds_20m_upscale['reflectance'] / intensity * panband )
    return sharpend

def stack_data(ds_10m, ds_20m_upscale, sharpend):
    # Creating the stacked (10m + upscaled 20m arrays)
    stacked_viewing_azimuth = xarray.concat([ds_20m_upscale['viewing_azimuth_grid'], ds_10m['viewing_azimuth_grid']], dim='band')
    stacked_viewing_zenith = xarray.concat([ds_20m_upscale['viewing_zenith_grid'], ds_10m['viewing_zenith_grid']], dim='band')
    stacked_reflectance = xarray.concat([sharpend, ds_10m['reflectance']], dim='band')

    # We need to drop the old reflectance + angle DA from the DS and re-set it with the stacked (now 10 band) DA. 
    # If we don't do this, the band coordinates do not get updated
    reflectance_attrs = ds_20m_upscale['reflectance'].attrs
    ds_20m_upscale = ds_20m_upscale.drop_vars('viewing_azimuth_grid')
    ds_20m_upscale = ds_20m_upscale.drop_vars('viewing_zenith_grid')
    ds_20m_upscale = ds_20m_upscale.drop_vars('reflectance').drop_vars('band')

    ds_20m_upscale['reflectance'] = stacked_reflectance
    ds_20m_upscale['reflectance'].attrs = reflectance_attrs
    ds_20m_upscale['viewing_zenith_grid'] = stacked_viewing_zenith
    ds_20m_upscale['viewing_azimuth_grid'] = stacked_viewing_azimuth
    ds_20m_upscale['band'] = ds_20m_upscale['band'].astype('str')
    ds_20m_upscale = ds_20m_upscale.chunk(band=-1, x=-1, y=1000, time=1)
    return ds_20m_upscale

def write_zarr(tile, ds_20m_upscale):    
    zarr_store = f'{prefix}/zarrs/{tile}_sharpend.zarr'
    ds_20m_upscale.to_zarr(zarr_store, mode='w', zarr_format=2)

def process_tile(tile):
    print(tile)
    ds_10m = read_10m(tile)
    ds_20m = read_20m(tile)
    print('finished reading data')
    ds_20m_upscale = upscale(ds_10m, ds_20m)
    print('finished scaling')
    sharpend = sharpen3(ds_10m, ds_20m_upscale)
    print('finished sharpening')
    ds_20m_upscale = stack_data(ds_10m, ds_20m_upscale, sharpend)
    print('finished stacking')
    write_zarr(tile, ds_20m_upscale)

In [5]:
tile = '11SLB'
ds_10m = read_10m(tile)
ds_20m = read_20m(tile)
ds_20m_upscale = upscale(ds_10m, ds_20m)
sharpend = sharpen3(ds_10m, ds_20m_upscale)
ds_20m_upscale = stack_data(ds_10m, ds_20m_upscale, sharpend)
ds_20m_upscale

/data/sentinel2//zarrs/11SLB_10m.zarr


<xarray.Dataset> Size: 5TB
Dimensions:               (time: 568, x_angles: 23, y_angles: 23, detector: 7,
                           x: 10980, y: 10980, band: 10)
Coordinates:
  * time                  (time) datetime64[ns] 5kB 2015-10-05T18:50:06.02800...
  * x_angles              (x_angles) float64 184B 3e+05 3.05e+05 ... 4.1e+05
  * y_angles              (y_angles) float64 184B 4.2e+06 4.195e+06 ... 4.09e+06
  * detector              (detector) <U2 56B '10' '11' '12' '6' '7' '8' '9'
  * x                     (x) float64 88kB 3e+05 3e+05 ... 4.098e+05 4.098e+05
  * y                     (y) float64 88kB 4.2e+06 4.2e+06 ... 4.09e+06 4.09e+06
  * band                  (band) <U3 120B 'B5' 'B6' 'B7' ... 'B3' 'B4' 'B8'
Data variables:
    sun_zenith_grid       (time, y_angles, x_angles) float32 1MB dask.array<chunksize=(1, 23, 23), meta=np.ndarray>
    AOT                   (time, y, x) float32 274GB dask.array<chunksize=(1, 1000, 10980), meta=np.ndarray>
    SCL                   (time, y, x) float64 548GB dask.array<chunksize=(1, 1000, 10980), meta=np.ndarray>
    sun_azimuth_grid      (time, y_angles, x_angles) float32 1MB dask.array<chunksize=(1, 23, 23), meta=np.ndarray>
    SNW                   (time, y, x) float64 548GB dask.array<chunksize=(1, 1000, 10980), meta=np.ndarray>
    meta                  (time) <U1 2kB dask.array<chunksize=(1,), meta=np.ndarray>
    CLD                   (time, y, x) float64 548GB dask.array<chunksize=(1, 1000, 10980), meta=np.ndarray>
    spatial_ref           int64 8B ...
    WVP                   (time, y, x) float32 274GB dask.array<chunksize=(1, 1000, 10980), meta=np.ndarray>
    reflectance           (band, time, y, x) float32 3TB dask.array<chunksize=(10, 1, 1000, 10980), meta=np.ndarray>
    viewing_zenith_grid   (time, band, detector, y_angles, x_angles) float32 84MB dask.array<chunksize=(1, 10, 7, 23, 23), meta=np.ndarray>
    viewing_azimuth_grid  (time, band, detector, y_angles, x_angles) float32 84MB dask.array<chunksize=(1, 10, 7, 23, 23), meta=np.ndarray>
Attributes: (12/84)
    AOT_QUANTIFICATION_VALUE:              1000.0
    AOT_QUANTIFICATION_VALUE_UNIT:         none
    AOT_RETRIEVAL_ACCURACY:                0.0
    AOT_RETRIEVAL_METHOD:                  CAMS
    BOA_QUANTIFICATION_VALUE:              10000
    BOA_QUANTIFICATION_VALUE_UNIT:         none
    ...                                    ...
    viewing_zenith_mean_B10:               5.47325314991262
    viewing_azimuth_mean_B10:              279.873263012713
    viewing_zenith_mean_B11:               5.57384161027337
    viewing_azimuth_mean_B11:              279.851879441611
    viewing_zenith_mean_B12:               5.67759180877617
    viewing_azimuth_mean_B12:              279.96581851259

In [ ]:
%%time
tile = '11SLB'
process_tile(tile)

In [ ]:
%%time
for tile in tiles[0:3]:
    process_tile(tile)

# Manual

## Load Data

In [ ]:
ds_10m = read_10m(tile)
ds_20m = read_20m(tile)
ds_20m_upscale = upscale(ds_10m, ds_20m)
sharpend = sharpen3(ds_10m, ds_20m_upscale)
ds_20m_upscale = stack_data(ds_10m, ds_20m_upscale, sharpend)
ds_20m_upscale = ds_20m_upscale.chunk(band=10, x=500, y=500)
ds_20m_upscale

In [ ]:
%%time 
zarr_store = f'/tablespace/sentinel2/{tile}_sharpend.zarr'
ds_20m_upscale.to_zarr(zarr_store, mode='w')

# Make Plots

In [ ]:
import matplotlib.pyplot as plt
date = '2024-02-25'

In [ ]:
def normalize(data):
    data = data - numpy.nanmin(data)
    data = data / numpy.nanmax(data)     
    return data

def mask(data):
    data = numpy.ma.masked_array(data, numpy.isnan(data))
    data[data.mask] = 0
    return data

In [ ]:
# Pandband
fig, ax = plt.subplots(dpi=300)

rgb = panband.sel(time=date).squeeze().transpose('y', 'x').values 

img = ax.imshow(rgb)
ax.axis('off')  
ax.set_aspect('equal')
cbar = fig.colorbar(img, ax=ax, shrink=0.7)
rgb.min(), rgb.max()

In [ ]:
# Intensity
fig, ax = plt.subplots(dpi=300)

rgb = intensity.sel(time=date).squeeze().transpose('y', 'x').values 
rgb = mask(rgb)

img = ax.imshow(rgb)
ax.axis('off')  
ax.set_aspect('equal')
cbar = fig.colorbar(img, ax=ax, shrink=0.7)
rgb.min(), rgb.max()

In [ ]:
# 10 m
fig, ax = plt.subplots(dpi=300)

rgb = ds_10m['reflectance'].sel(time=date, band=['B4', 'B3', 'B2']).squeeze().transpose('y', 'x', 'band').values 
rgb = mask(normalize(rgb))

ax.imshow(rgb)
ax.axis('off')  
ax.set_aspect('equal')

In [ ]:
# 20 m
fig, ax = plt.subplots(dpi=300)

rgb = ds_20m['reflectance'].sel(time=date, band=['B8A', 'B5', 'B6']).squeeze().transpose('y', 'x', 'band').values 
#rgb = normalize(rgb)

plt.imshow(rgb)
plt.axis('off')  
ax.set_aspect('equal')
rgb.min(), rgb.max()

In [ ]:
# 20 m Sharpend
fig, ax = plt.subplots(dpi=300)

rgb = ds_20m_upscale['reflectance'].sel(time=date, band=['B8A', 'B5', 'B6']).squeeze().transpose('y', 'x', 'band').values 
rgb = mask(rgb)

plt.imshow(rgb)
plt.axis('off')  
ax.set_aspect('equal')
rgb.min(), rgb.max()

# Make geotiffs

In [ ]:
ds_10m.rio.write_crs(32611, inplace=True)
ds_10m.rio.set_spatial_dims('x', 'y', inplace=True)
ds_10m['reflectance'].sel(time=date).squeeze().to_dataset('band').rio.to_raster(f'{date}_10m.tiff')

ds_20m.rio.write_crs(32611, inplace=True)
ds_20m.rio.set_spatial_dims('x', 'y', inplace=True)
ds_20m['reflectance'].sel(time=date).squeeze().to_dataset('band').rio.to_raster(f'{date}_20m.tiff')

ds_20m_upscale.rio.write_crs(32611, inplace=True)
ds_20m_upscale.rio.set_spatial_dims('x', 'y', inplace=True)
ds_20m_upscale['reflectance'].sel(time=date).squeeze().to_dataset('band').rio.to_raster(f'{date}_sharpend.tiff')

# Some testing

In [ ]:
max = ds_20m['reflectance'][:, 1, :, :].max(dim=('x', 'y'))
max = max.compute()

In [ ]:
plt.plot(max['time'], max)
plt.xlabel('time')
plt.ylabel('Max value')
plt.title('Maximum values across dim2, dim3, dim4')
plt.show()